In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

#load data
num_train = 49000
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size = 64,
                                           sampler = torch.utils.data.sampler.SubsetRandomSampler(range(num_train)))
#valset = torchvision.dataset.CIFAR10(root = './data', train = True, download = True, transform = transform)
val_loader = torch.utils.data.DataLoader(trainset, batch_size = 64,
                                         sampler = torch.utils.data.sampler.SubsetRandomSampler(range(num_train,50000)))
testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size = 64)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
#set device to gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5,padding = 2)
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,16,3,padding = 1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(8*8*16,10)
    
    def forward(self,x):
        scores = self.pool1(F.relu(self.conv1(x)))
        scores = self.pool2(F.relu(self.conv2(self.bn(scores))))
        scores = scores.view(-1,8*8*16)
        scores = self.fc(scores)
        return scores

In [5]:
def check_acc(loader,model):
    if loader.dataset.train:
        print('checking on validation set')
    else:
        print('checking on test set')
        
    num_correct = 0
    num_sample = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            scores = model(x)
            _,pred = torch.max(scores,1)
            num_correct += (pred == y).sum()
            num_sample += y.size(0)
        acc = float(num_correct)/num_sample
        print('acc = %.2f' % (acc*100))
    

In [9]:
print_every = 100
def trainNet(model, optimizer, epoch):
    model = model.to(device=device)
    for e in range(epoch):
        for t,(x,y) in enumerate(train_loader):
            model.train()
            x = x.to(device = device)
            y = y.to(device = device)
            
            scores = model(x)
            print(scores.size())
            loss = F.cross_entropy(scores,y)
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()
            
            if t%print_every == 0:
                print('Iteration = %d, loss = %4f' % (t,loss.item()))
                check_acc(val_loader,model)
                print()
        

In [10]:
import torch.optim as optim
model = Net()
model = nn.DataParallel(model)#GPU并行运算
optimizier1 = optim.SGD(model.parameters(),lr = 2e-3)

trainNet(model, optimizier1, 4)



torch.Size([64, 10])
Iteration = 0, loss = 2.394830
checking on validation set
acc = 13.70

torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])
torch

KeyboardInterrupt: 

In [10]:
import numpy as np
lr = 10 ** np.random.uniform(-6,1,10)
print(lr)

[1.05333837e-06 3.97712630e-04 1.42084458e-03 9.77416092e-03
 1.04650263e-02 1.41657990e-06 1.84572252e-02 1.16714934e-01
 3.34535455e-04 7.86219624e-03]
